In [175]:
import pandas as pd
import numpy as np


In [160]:
from pandas.io.json import json_normalize

#Get Toronto Neighborhoods from wiki

In [105]:
#import BeautifulSoup
from bs4 import BeautifulSoup
import requests

In [106]:
#get url
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

# Parse the html content
soup = BeautifulSoup(html_content, "lxml")

In [107]:
#find html table
html_table = soup.find("table", attrs={"class": "wikitable"})

#get all rows of the table
table_row_list = html_table.tbody.find_all("tr")

In [108]:
TorontoDF = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

for i in range(0, len(table_row_list)):
  td_list = table_row_list[i].find_all("td")
  if len(td_list)>0:
    if td_list[1].text.replace('\n', '')!='Not assigned':      
      TorontoDF = TorontoDF.append({'PostalCode': td_list[0].text.replace('\n', ''), 'Borough': td_list[1].text.replace('\n', ''), 'Neighborhood': td_list[2].text.replace('\n', '')}, ignore_index=True)


In [109]:
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [110]:
TorontoDF.shape

(103, 3)

In [111]:
#TorontoDF.to_csv('TorontoNetghb')

#Get Toronto postal codes

In [112]:
#TorontoDF = pd.read_csv('TorontoNetghb')

In [113]:
#TorontoDF.drop(['Unnamed: 0'], axis='columns', inplace=True)

In [114]:
TorontoDF.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [115]:
codes = pd.read_csv('http://cocl.us/Geospatial_data')

In [116]:
codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [117]:
codes=codes.rename(columns={'Postal Code': 'PostalCode'})

In [118]:
codes.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [119]:
TorontoDF = TorontoDF.merge(codes)

In [120]:
TorontoDF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#Explore and cluster the neighborhoods in Toronto

In [128]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [121]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(TorontoDF['Borough'].unique()),
        TorontoDF.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


Create a map of Toronto with neighborhoods

In [153]:
Central = TorontoDF[TorontoDF['Neighborhood'] == 'Davisville North']
latitude = Central.iloc[0]['Latitude']
longitude = Central.iloc[0]['Longitude']

In [154]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoDF['Latitude'], TorontoDF['Longitude'], TorontoDF['Borough'], TorontoDF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [134]:
Central_data = TorontoDF[TorontoDF['Borough'] == 'Central Toronto'].reset_index(drop=True)
Central_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


In [155]:
# create map of Toronto using latitude and longitude values
map_Central_Toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(Central_data['Latitude'], Central_data['Longitude'], Central_data['Borough'], Central_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='green',
        fill=False,
        fill_color='#cc86cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_Central_Toronto)  
    
map_Central_Toronto

In [133]:
CLIENT_ID = 'WWKCYIA5GJWJFZUPW5NRECC0RZBSUTGAM5SKTH0GU1T315DV' # your Foursquare ID
CLIENT_SECRET = 'AZGYWNMIYLUA1EZ1WYFAOMITCV3IWRXSMXBWY4L4LXWKAKS5' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: WWKCYIA5GJWJFZUPW5NRECC0RZBSUTGAM5SKTH0GU1T315DV
CLIENT_SECRET:AZGYWNMIYLUA1EZ1WYFAOMITCV3IWRXSMXBWY4L4LXWKAKS5


let's get the top 100 venues that are in Davisville North within a radius of 500 meters.

In [156]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=WWKCYIA5GJWJFZUPW5NRECC0RZBSUTGAM5SKTH0GU1T315DV&client_secret=AZGYWNMIYLUA1EZ1WYFAOMITCV3IWRXSMXBWY4L4LXWKAKS5&v=20180604&ll=43.7127511,-79.3901975&radius=500&limit=100'

In [157]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa47fef6e2aa02db461cd90'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4adb2fd3f964a520c42421e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/breakfast_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d143941735',
         'name': 'Breakfast Spot',
         'pluralName': 'Breakfast Spots',
         'primary': True,
         'shortName': 'Breakfast'}],
       'id': '4adb2fd3f964a520c42421e3',
       'location': {'address': '955 Mount Pleasant',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Mount Pleasant & Erskine',
        'distance': 110,
        'formattedAddress': ['955 Mount Pleasant (Mount Pleasant & Erskine)',
         'Toronto ON M4P 2L7',
         '

In [158]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [161]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
1,Sherwood Park,Park,43.716551,-79.387776
2,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
3,Winners,Department Store,43.713236,-79.393873
4,Best Western Roehampton Hotel & Suites,Hotel,43.708878,-79.390880


In [162]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [164]:
Central_Toronto_venues = getNearbyVenues(names=Central_data['Neighborhood'],
                                   latitudes=Central_data['Latitude'],
                                   longitudes=Central_data['Longitude']
                                  )

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [165]:
print(Central_Toronto_venues.shape)
Central_Toronto_venues.head()

(104, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
4,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue


In [167]:
Central_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,9,9,9,9,9,9
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
"North Toronto West, Lawrence Park",18,18,18,18,18,18
Roselawn,2,2,2,2,2,2
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",14,14,14,14,14,14
"The Annex, North Midtown, Yorkville",19,19,19,19,19,19


In [168]:
print('There are {} uniques categories.'.format(len(Central_Toronto_venues['Venue Category'].unique())))

There are 60 uniques categories.


In [169]:
# one hot encoding
Central_Toronto_onehot = pd.get_dummies(Central_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Central_Toronto_onehot['Neighborhood'] = Central_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Central_Toronto_onehot.columns[-1]] + list(Central_Toronto_onehot.columns[:-1])
Central_Toronto_onehot = Central_Toronto_onehot[fixed_columns]

Central_Toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [171]:
Central_Toronto_grouped = Central_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Central_Toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Dance Studio,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Park,Pharmacy,Pizza Place,Playground,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.060606,0.000000,0.000000,0.060606,0.000000,0.000000,0.090909,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.0,0.030303,0.030303,0.030303,0.060606,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.060606,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.030303,0.030303,0.090909,0.0,0.000000,0.030303,0.000000,0.090909,0.030303,0.000000,0.000000,0.000000,0.060606,0.000000,0.030303,0.030303,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.50,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.055556,0.111111,0.111111,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.055556,0.000000,0.0,0.055556,0.000000,0.000000,0.0,0.000000,0.055556,0.055556,0

In [172]:
num_top_venues = 5

for hood in Central_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Central_Toronto_grouped[Central_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Pizza Place  0.09
1        Dessert Shop  0.09
2      Sandwich Place  0.09
3  Italian Restaurant  0.06
4                 Gym  0.06


----Davisville North----
                  venue  freq
0                 Hotel  0.11
1          Dance Studio  0.11
2  Gym / Fitness Center  0.11
3                  Park  0.11
4     Food & Drink Shop  0.11


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0                Trail  0.25
1        Jewelry Store  0.25
2     Sushi Restaurant  0.25
3                 Park  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0          Swim School  0.33
1             Bus Line  0.33
2                 Park  0.33
3  American Restaurant  0.00
4                  Pub  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Playground   0.5
1                Trail   0.5
2  American Restaurant   0.0
3    Indian Restaurant

In [173]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [177]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Central_Toronto_grouped['Neighborhood']

for ind in np.arange(Central_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Central_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Pizza Place,Sandwich Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Gym,Farmers Market,Indian Restaurant
1,Davisville North,Gym / Fitness Center,Park,Department Store,Dance Studio,Sandwich Place,Food & Drink Shop,Dog Run,Breakfast Spot,Hotel,Garden
2,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Trail,Sushi Restaurant,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
3,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Diner,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Furniture / Home Store
4,"Moore Park, Summerhill East",Playground,Trail,Department Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop


In [178]:
# set number of clusters
kclusters = 5

Central_Toronto_grouped_clustering = Central_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Central_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 0, 4, 1, 3, 1, 1], dtype=int32)

In [182]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Central_Toronto_merged = Central_data

# merge Central_Toronto_grouped with Central_data to add latitude/longitude for each neighborhood
Central_Toronto_merged = Central_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Central_Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Swim School,Bus Line,Park,Yoga Studio,Diner,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Furniture / Home Store
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,3,Garden,Music Venue,Yoga Studio,Gym / Fitness Center,Greek Restaurant,Gourmet Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym / Fitness Center,Park,Department Store,Dance Studio,Sandwich Place,Food & Drink Shop,Dog Run,Breakfast Spot,Hotel,Garden
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Diner,Dog Run,Donut Shop,Farmers Market,Fast Food Restaurant,Yoga Studio
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Coffee Shop,Clothing Store,Salon / Barbershop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant


In [185]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_Toronto_merged['Latitude'], Central_Toronto_merged['Longitude'], Central_Toronto_merged['Neighborhood'], Central_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters